<a href="https://colab.research.google.com/github/VictorCastroDeFaria/LinearProgramming/blob/main/ProblemasLista03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pulp

import pulp
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 8.0 MB/s 


In [ ]:
departing_cities = list(range(5))
arriving_cities = list(range(10))
routes = [(i, j) for i in departing_cities for j in arriving_cities]

# Variáveis de decisão
x = pulp.LpVariable.dicts('Units', (departing_cities,arriving_cities), lowBound = 0, cat = 'Linear')
y = pulp.LpVariable.dicts('Suppliance', (departing_cities,arriving_cities), lowBound = 0, cat = 'Binary')

minimum_supplies = [3, 2, 3, 2, 3, 2, 2, 3, 3, 3]
demand = [100, 65, 100, 70, 120, 60, 75, 100, 95, 85]
minimum_delivery = [25, 30, 30, 35, 25]
capacity = [175, 175, 175, 175, 175]

departing_strings = ['Seattle','São Francisco','Las Vegas','Phoenix','Denver']
arriving_strings = ['Washington','Oregon','California','Idaho','Nevada','Montana','Wyoming','Arizona','Utah','Colorado']

delivery_costs = [[10, 15, 10, 15, 20, 20, 20, 40, 10, 30],
                  [30, 15, 10, 20, 10, 20, 20, 30, 20, 30],
                  [20, 10, 5, 15, 10, 15, 15, 10, 5, 5],
                  [40, 25, 15, 20, 10, 30, 30, 10, 15, 10],
                  [30, 30, 25, 10, 5, 35, 35, 15, 5, 10]]
#print(routes)
#print(type(x))
#print(x.keys())

#for (i,j) in routes:
#  route = (i,j)
#  print(delivery_costs[i][j], minimum_supplies[j], demand[j], minimum_delivery[i], capacity[i])
#  print(x[i][j])
# Objetivo: Minimizar os custos de entrega de todas as unidades, de maneira a satisfazer a demanda mínima
problem = LpProblem("Problema da minimização de custos", LpMinimize)

problem += lpSum([x[i][j]*delivery_costs[i][j] for (i,j) in routes])

# Existe um limite mínimo de fornecedores para cada uma das cidades de chegada (função soma das colunas da matriz Y)
for j in arriving_cities:
  problem += lpSum([y[i][j] for i in departing_cities]) >= minimum_supplies[j]
# Deve ser atendida a demanda mínima para cada uma das cidades de chegada (Função soma das colunas da matriz X)
for j in arriving_cities:
  problem += lpSum([x[i][j] for i in departing_cities]) >= demand[j]
# A capacidade de armazenamento para cada cidade de partida deve ser inferior ao limite máximo (função soma das linhas de X)
for i in departing_cities:
  problem += lpSum([x[i][j] for j in arriving_cities]) <= capacity[i]
# Cada fornecedor deve entregar um mínimo de unidades de produtos, se houver entrega
for (i,j) in routes:
  problem += x[i][j] >= minimum_delivery[i]*y[i][j]
# Restrição big-M para as variáveis y e x (yij deve ser não nulo se e somente se xij for não nulo, isto é, xij <= M*yij)
big_M = 1000000
for (i,j) in routes:
  problem += x[i][j] <= big_M*y[i][j]

problem.solve()

print("Status:", LpStatus[problem.status])

print("objective = %s" % value(problem.objective))
print('----------------------------')
print("Matriz X:")
for i in departing_cities:
  for j in arriving_cities:
    print(int(x[i][j].varValue),end = ' ')
  print('')
print('----------------------------')

print("Matriz Y:")
for i in departing_cities:
  for j in arriving_cities:
    print(int(y[i][j].varValue),end = ' ')
  print('')
print('----------------------------')

print("Impressão da soma de localidades de chegadas supridas para cada localidade de partida")
print(" ")
for i in departing_cities:
  print(departing_strings[i],": ",sum(y[i][j].varValue for j in arriving_cities))
print('----------------------------')

print("Impressão da soma de fornecedores para cada localidade de chegada")
print(" ")
for j in arriving_cities:
  print(arriving_strings[j],": ",sum(y[i][j].varValue for i in departing_cities))

print('----------------------------')

print("Impressão de todas as entregas que saem de cada cidade de partida")
print(" ")
for i in departing_cities:
  print(departing_strings[i],": ",sum(x[i][j].varValue for j in arriving_cities))
print('----------------------------')

print("Impressão de todas as entregas para cada localidade de chegada")
print(" ")
for j in arriving_cities: # Imprimindo cada entrega
  print(arriving_strings[j],": ",sum(x[i][j].varValue for i in departing_cities))
print('----------------------------')

print("Impressão dos custos parciais para cada localidade de chegada")
print(" ")
for j in arriving_cities:
  print(arriving_strings[j],": ",sum(x[i][j].varValue*delivery_costs[i][j] for i in departing_cities))
print('----------------------------')

Status: Optimal
objective = 10800.0
----------------------------
Matriz X:
45 25 25 30 0 0 25 0 25 0 
0 30 30 0 35 30 50 0 0 0 
30 10 10 0 0 30 0 30 35 30 
0 0 35 0 60 0 0 45 0 35 
25 0 0 40 25 0 0 25 35 25 
----------------------------
Matriz Y:
1 1 1 1 0 0 1 0 1 0 
0 1 1 0 1 1 1 0 0 0 
1 0 0 0 0 1 0 1 1 1 
0 0 1 0 1 0 0 1 0 1 
1 0 0 1 1 0 0 1 1 1 
----------------------------
Impressão da soma de localidades de chegadas supridas para cada localidade de partida
 
Seattle :  6.0
São Francisco :  5.0
Las Vegas :  5.0
Phoenix :  4.0
Denver :  6.0
----------------------------
Impressão da soma de fornecedores para cada localidade de chegada
 
Washington :  3.0
Oregon :  2.0
California :  3.0
Idaho :  2.0
Nevada :  3.0
Montana :  2.0
Wyoming :  2.0
Arizona :  3.0
Utah :  3.0
Colorado :  3.0
----------------------------
Impressão de todas as entregas que saem de cada cidade de partida
 
Seattle :  175.0
São Francisco :  175.0
Las Vegas :  175.0
Phoenix :  175.0
Denver :  175.0
-------------

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
